In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pysentani as sti

In [2]:
survey = pd.read_excel('../data-anonymous/sentani-merged-cleaned-anonymous-2014-11-29.xlsx')
# pysentani access type function
survey['access_type'] = sti.access_type(survey)
# selecting columns to analyze
survey = survey[['app_now/rice_cooker',
                 'app_now/TV',
                 'app_now/fridge',
                 'app_now/fan',
                 'app_now/lighting',
                 'app_now/radio',
                 'app_buy/rice_cooker',
                 'app_buy/TV',
                 'app_buy/fridge',
                 'app_buy/fan',
                 'app_buy/lighting',
                 'app_buy/radio',
                 'access_type',]]
# dropping all NaN values for above columns
# we don't need to do these per appliance because the nulls are correlated
survey = survey.dropna()

In [3]:
survey.mean()

app_now/rice_cooker    0.163373
app_now/TV             0.810277
app_now/fridge         0.180501
app_now/fan            0.117260
app_now/lighting       0.928854
app_now/radio          0.349144
app_buy/rice_cooker    0.246377
app_buy/TV             0.202899
app_buy/fridge         0.476943
app_buy/fan            0.185771
app_buy/lighting       0.256917
app_buy/radio          0.110672
dtype: float64

In [4]:
def willing_to_buy(row, now=None, buy=None):
    if row[now]==1:
        return np.nan
    if row[now]==0 and row[buy]==0:
        return 0
    if row[now]==0 and row[buy]==1:
        return 1
    

def calc_willingness(survey, appliance):
    app_now_appliance = 'app_now/' + appliance
    app_buy_appliance = 'app_buy/' + appliance
    # when we average this, do we exclude the folks with the appliance?
    #survey['wb/' + appliance] = ~survey[app_now_appliance].astype(bool) & survey[app_buy_appliance].astype(bool)
    survey['wb/' + appliance] = survey.apply(willing_to_buy, 
                                axis=1, 
                                now=app_now_appliance,
                                buy=app_buy_appliance)
    
    
appliance_list = ['TV', 'lighting', 'radio', 'rice_cooker', 'fridge', 'fan' ]

# Creating variable with for loop adding each from appliance_list to calc_willingess function
WBlist = []
for appliance in appliance_list:
    calc_willingness(survey, appliance)
    WBlist.append('wb/' + appliance)

In [5]:
survey.head()

,app_now/rice_cooker,app_now/TV,app_now/fridge,app_now/fan,app_now/lighting,app_now/radio,app_buy/rice_cooker,app_buy/TV,app_buy/fridge,app_buy/fan,app_buy/lighting,app_buy/radio,access_type,wb/TV,wb/lighting,wb/radio,wb/rice_cooker,wb/fridge,wb/fan
0,0,1,0,0,0,1,0,0,1,0,0,0,no_access,NaN,0,NaN,0,1,0
2,0,1,0,0,0,0,0,0,1,1,0,0,no_access,NaN,0,0,0,1,1
4,0,0,0,0,1,0,1,1,1,1,0,1,no_access,1,NaN,1,1,1,1
5,0,1,0,0,1,0,0,0,1,0,0,0,no_access,NaN,NaN,0,0,1,0
6,0,1,0,0,1,0,0,1,0,0,0,0,no_access,NaN,NaN,0,0,0,0


In [6]:
subsurvey_now = survey[['app_now/rice_cooker',
                 'app_now/TV',
                 'app_now/fridge',
                 'app_now/fan',
                 'app_now/lighting',
                 'app_now/radio',
                 'access_type']]
analyzed_now = subsurvey_now.groupby('access_type').mean()
#analyzed_now['Access Type'] = analyzed_now.index 
analyzed_now_multiindex = analyzed_now.stack()
analyzed_now_multiindex

access_type                             
PLN_grid             app_now/rice_cooker    0.270983
                     app_now/TV             0.798561
                     app_now/fridge         0.314149
                     app_now/fan            0.153477
                     app_now/lighting       0.928058
                     app_now/radio          0.304556
PLN_microgrid        app_now/rice_cooker    0.017391
                     app_now/TV             0.808696
                     app_now/fridge         0.026087
                     app_now/fan            0.026087
                     app_now/lighting       0.965217
                     app_now/radio          0.295652
community_microgrid  app_now/rice_cooker    0.133333
                     app_now/TV             0.933333
                     app_now/fridge         0.022222
                     app_now/fan            0.088889
                     app_now/lighting       0.911111
                     app_now/radio          0.511111
no_ac

In [7]:
# new dataframe classifying WBlist by access type
subsurvey_buy = survey[['access_type'] + WBlist]
# drop surv
analyzed_buy = subsurvey_buy.groupby('access_type').mean()
#analyzed_buy['Access Type'] = analyzed_buy.index 
analyzed_buy_multiindex = analyzed_buy.stack()
analyzed_buy_multiindex.index
analyzed_buy_multiindex

access_type                        
PLN_grid             wb/TV             0.726190
                     wb/lighting       0.133333
                     wb/radio          0.072414
                     wb/rice_cooker    0.305921
                     wb/fridge         0.727273
                     wb/fan            0.192635
PLN_microgrid        wb/TV             0.636364
                     wb/lighting       0.000000
                     wb/radio          0.074074
                     wb/rice_cooker    0.194690
                     wb/fridge         0.196429
                     wb/fan            0.080357
community_microgrid  wb/TV             1.000000
                     wb/lighting       0.250000
                     wb/radio          0.318182
                     wb/rice_cooker    0.410256
                     wb/fridge         0.409091
                     wb/fan            0.390244
no_access            wb/TV             0.742857
                     wb/lighting       0.062500
    

In [8]:
future_electrification = analyzed_buy_multiindex + analyzed_now_multiindex 

In [9]:
future_electrification

access_type                             
PLN_grid             app_now/TV            NaN
                     app_now/fan           NaN
                     app_now/fridge        NaN
                     app_now/lighting      NaN
                     app_now/radio         NaN
                     app_now/rice_cooker   NaN
                     wb/TV                 NaN
                     wb/fan                NaN
                     wb/fridge             NaN
                     wb/lighting           NaN
                     wb/radio              NaN
                     wb/rice_cooker        NaN
PLN_microgrid        app_now/TV            NaN
                     app_now/fan           NaN
                     app_now/fridge        NaN
                     app_now/lighting      NaN
                     app_now/radio         NaN
                     app_now/rice_cooker   NaN
                     wb/TV                 NaN
                     wb/fan                NaN
                   

In [10]:
# renaming subcategories of Access Type
survey.replace(to_replace = {'access_type': {'PLN_grid': 'PLN Grid',
                                             'community_microgrid': 'Community Microgrid',
                                             'PLN_microgrid': 'PLN Microgrid',
                                             'no_access': 'No Access'}},inplace = True)

survey.rename(columns={'app_now/rice_cooker': 'app_now/Rice Cooker',
                       'app_now/TV': 'app_now/TV',
                       'app_now/fridge': 'app_now/Fridge',
                       'app_now/fan':'app_now/Fan',
                       'app_now/lighting': 'app_now/Lighting',
                       'app_now/radio': 'app_now/Radio',
                       'app_buy/rice_cooker': 'app_buy/Rice Cooker',
                       'app_buy/TV': 'app_buy/TV',
                       'app_buy/fridge': 'app_buy/Fridge',
                       'app_buy/fan':'app_buy/Fan',
                       'app_buy/lighting': 'app_buy/Lighting',
                       'app_buy/radio': 'app_buy/Radio',
                       'access_type': 'Access Type'},
                                 inplace = True)



In [18]:
survey[survey['app_now/TV']==0]['app_now/Fridge']

4       0
13      0
24      0
29      0
36      0
41      0
52      0
56      0
63      0
84      0
92      0
112     1
113     0
123     0
130     0
142     0
168     0
175     1
187     0
195     0
209     0
217     0
219     0
269     0
284     0
286     0
287     0
289     0
290     0
297     0
       ..
907     0
914     0
916     0
917     0
933     0
947     0
953     0
978     0
984     0
985     0
997     0
1000    0
1016    1
1017    0
1020    0
1037    0
1039    1
1040    1
1041    0
1054    0
1058    0
1063    0
1078    0
1084    0
1098    1
1104    0
1151    1
1164    0
1178    0
1182    0
Name: app_now/Fridge, dtype: float64

In [21]:
def willing_to_buy(own, want):
    if own==1:
        return np.nan
    if own==0 and want==0:
        return 0
    if own==0 and want==1:
        return 1
        

In [22]:
willing_to_buy(0,0)

0

In [24]:
willing_to_buy(1,1)

nan

In [25]:
willing_to_buy(1,0)

nan

In [26]:
willing_to_buy(0,1)

1

In [31]:
survey

,app_now/rice_cooker,app_now/TV,app_now/fridge,app_now/fan,app_now/lighting,app_now/radio,app_buy/rice_cooker,app_buy/TV,app_buy/fridge,app_buy/fan,app_buy/lighting,app_buy/radio,access_type
0,0,1,0,0,0,1,0,0,1,0,0,0,no_access
2,0,1,0,0,0,0,0,0,1,1,0,0,no_access
4,0,0,0,0,1,0,1,1,1,1,0,1,no_access
5,0,1,0,0,1,0,0,0,1,0,0,0,no_access
6,0,1,0,0,1,0,0,1,0,0,0,0,no_access
9,0,1,0,0,0,1,1,0,1,1,0,0,no_access
10,0,1,0,0,1,0,0,0,0,0,1,0,no_access
11,0,1,0,0,1,0,1,1,1,1,0,0,no_access
12,0,1,0,0,1,0,1,0,0,0,0,0,no_access
13,0,0,0,0,1,0,0,1,0,0,0,0,no_access


In [32]:
def willing_to_buy(row):
    if row['app_now/lighting']==1:
        return np.nan
    if row['app_now/lighting']==0 and row['app_buy/lighting']==0:
        return 0
    if row['app_now/lighting']==0 and row['app_buy/lighting']==1:
        return 1
    
survey['test'] = survey.apply(willing_to_buy, axis=1)

In [35]:
survey['test'].value_counts(dropna=False)

NaN    705
 0      48
 1       6
Name: test, dtype: int64

In [41]:
def willing_to_buy(row, now=None, buy=None):
    if row[now]==1:
        return np.nan
    if row[now]==0 and row[buy]==0:
        return 0
    if row[now]==0 and row[buy]==1:
        return 1
    
survey['test'] = survey.apply(willing_to_buy, 
                              axis=1, 
                              now='app_now/TV',
                              buy='app_buy/TV')

In [42]:
survey['test'].value_counts(dropna=False)

NaN    615
 1     104
 0      40
Name: test, dtype: int64